In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Activation
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import seaborn as sns
import cv2
import os

import optuna
import wandb 
import gc

In [ ]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    print("TensorFlow is using the GPU \n", gpus)
else:
    print("No GPU detected.")
    
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
gc.collect()
tf.keras.backend.clear_session()

In [ ]:
from wandb.integration.keras import WandbMetricsLogger

wandb.require("core")
wandb.login()

In [ ]:
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, (160, 160))
    return img, label

def make_dataset(paths, labels, batch_size = 32, shuffle = False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(10000)
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
# Se cargan los datos
# Se debe de realizar una transformación en los datos, haciendo que todas las identidades sean 0:

ds = pd.read_csv("/tf/Face-Recognition/CelebA/identity_CelebA.txt", sep = r"\s+", names=["image", "identity"])

ds["identity"] = 0
ds.head()

In [ ]:
# img_dir = "/tf/Face-Recognition/CelebA/img_align_celeba"

# ds["image_path"] = ds["image"].apply(lambda x: os.path.join(img_dir, x))
ds = ds.sample(4000, random_state = 4).reset_index(drop = True)
ds.head()

In [ ]:
image = np.array([])
identity = np.zeros(40)

for i in range(40):
    image = np.append(image, f"me{i+1}.jpg")
    identity[i] = 1

identity = identity.astype(int)

# print(image)
# print(identity)

df = pd.DataFrame({
    "image": image,
    "identity": identity,
})

# me_dir = "/tf/Face-Recognition/CelebA/Me"
# df["image_path"] = df["image"].apply(lambda x: os.path.join(me_dir, x))

df.head()

In [ ]:
ds = pd.concat([ds, df], axis = 0)
ds = ds.sample(frac = 1, random_state = 5).reset_index(drop = True)
ds

In [ ]:
df_train, df_temp = train_test_split(
    ds, test_size = 0.2, stratify = ds["identity"], random_state = 5)

df_val, df_test = train_test_split(
    df_temp, test_size = 0.5, stratify = df_temp["identity"], random_state = 5)

In [ ]:
print(df_train["identity"].value_counts())
print(df_val["identity"].value_counts())
print(df_test["identity"].value_counts())

In [ ]:
import shutil

base = "binary_faces"

for split in ["train", "val", "test"]:
    os.makedirs(f"{base}/{split}/me", exist_ok = True)
    os.makedirs(f"{base}/{split}/others", exist_ok = True)

In [ ]:
def copy_files(df, split):
    for idx, row in df.iterrows():
        if row["identity"] == 1:
            src = os.path.join("/tf/Face-Recognition/CelebA/Me", row["image"])
            dst = f"{base}/{split}/me/{row['image']}"
        else:
            src = os.path.join("/tf/Face-Recognition/CelebA/img_align_celeba", row["image"])
            dst = f"{base}/{split}/others/{row['image']}"
        
        shutil.copy(src, dst)

In [ ]:
copy_files(df_train, "train")
copy_files(df_val, "val")
copy_files(df_test, "test")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = 0.2,
    horizontal_flip = True)

test_val_datagen = ImageDataGenerator()

In [ ]:
train = train_datagen.flow_from_directory(
    f"{base}/train",
    target_size = (160,160),
    class_mode = "binary",
    batch_size = 32,
    seed = 4)

val = test_val_datagen.flow_from_directory(
    f"{base}/val",
    target_size = (160,160),
    class_mode = "binary",
    batch_size = 32,
    seed = 4)

test = test_val_datagen.flow_from_directory(
    f"{base}/test",
    target_size = (160,160),
    class_mode = "binary",
    batch_size = 32,
    seed = 4)

In [ ]:
FeatureExtractor = tf.keras.models.load_model("/tf/Face-Recognition/Models/Conv2D-MobileNetV2-Based-Fine-Tunned.keras")
FeatureExtractor.trainable = False

FeatureExtractor_Output = FeatureExtractor.layers[-3].output
FeatureExtractor_Model = tf.keras.Model(FeatureExtractor.input, FeatureExtractor_Output)

In [ ]:
lr = 1e-4
optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr) 

In [ ]:
inputs = tf.keras.Input(shape=(160,160,3))
x = FeatureExtractor_Model(inputs, training = False)
x = tf.keras.layers.Dense(256, activation="leaky_relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation = "leaky_relu")(x)
x = tf.keras.layers.Dropout(0.15)(x)
x = tf.keras.layers.Dense(64, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(1, activation = "sigmoid", dtype = "float32")(x)
model = tf.keras.Model(inputs, outputs)

model.compile(loss = "binary_crossentropy",
              optimizer = optimizer,
              metrics = ["accuracy"])
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor = "val_accuracy", patience = 10, restore_best_weights = True)
lr_reduction = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 7)

In [ ]:
wandb.init(
        project = "Face-Recognition-Conv2D-Trials-Exp-Series1.0",
        name = "Trial_1_FullSet",
        reinit = True,
        config = {
            "activation": "leaky_relu, relu",
            "n_layers": 3,
            "learning_rate": lr,
            "optimizer": "RMSProp"
        }
    )

In [ ]:
history = model.fit(
    train, 
    validation_data = val,
    epochs = 200,
    verbose = 1, 
    callbacks = [WandbMetricsLogger(log_freq = 5), early_stopping, lr_reduction]
        )

In [ ]:
model.save("Face-Recognition-Conv2D.keras")

In [ ]:
tf.keras.backend.clear_session()
wandb.finish()
gc.collect()